In [ ]:
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
with open('/content/drive/MyDrive/ai chatbots/intent.json') as file:
    data = json.load(file)
print(data)
training_sentences = []
training_labels = []
labels = []
responses = []

for intent in data['intents']:
    for pattern in intent['questions']:
        training_sentences.append(pattern)
        training_labels.append(intent['intent'])
    responses.append(intent['responses'])

    if intent['intent'] not in labels:
        labels.append(intent['intent'])

num_classes = len(labels)

# convert target labels
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
#print(lbl_encoder)
#print(training_labels)
training_labels = lbl_encoder.transform(training_labels)
#print(training_labels)

# vectorize data corpus
vocab_size = 1000
embedding_dim = 16
max_len = 20
# deal with out of token
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
# make all training sequences same size
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

# define nn
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.summary()

# train the model
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs,verbose = 0)

model.save("chat_model")

import pickle

# to save fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

{'intents': [{'intent': 'greeting', 'questions': ['Hi', 'Hey', 'Is anyone there?', 'Hello', 'Hay', 'Yo', 'Whatsup', 'Hello, Course Creator', 'Hi there', 'Hey, CurricuBot', 'Greetings, CurricuBot'], 'responses': ['Hello', 'Hi', 'Hi there', 'Hey!', 'Welcome!', "Hello! I'm Course Creator, your course planning bot. How can I assist you today?", 'Hello! Welcome to CurricuBot!', 'Hi! Welcome to CurricuBot!', 'Hi there!', 'Hi! How can I help you?', 'Hi! Is there something that I can do for you?']}, {'intent': 'asking', 'questions': ['How are you?'], 'responses': ['I am good. Thank you! How may I help you?']}, {'intent': 'goodbye', 'questions': ['Bye', 'See you later', 'Goodbye'], 'responses': ['See you later', 'Have a nice day', 'Bye! Come back again']}, {'intent': 'thanks', 'questions': ['Thanks', 'Thank you', "That's helpful", 'Thanks for the help'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure', "You're most welcome!"]}, {'intent': 'about', 'questions': ['Who are you?', 'What 

In [ ]:
import json
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import random
import pickle

with open("/content/drive/MyDrive/ai chatbots/intent.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20

    while True:
        inp = input("User: ")
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len),verbose = 0)

        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['intent'] == tag:
                print("User: ", inp)
                print("ChatBot: ", np.random.choice(i['responses']))

In [ ]:
print("Start messaging with the bot (type quit to stop)!")
chat()

Start messaging with the bot (type quit to stop)!
User: bn
User:  bn
ChatBot:  You can mail any problem you experience in this mail:  _______@gmail.com
User: bn
User:  bn
ChatBot:  You can mail any problem you experience in this mail:  _______@gmail.com
User: ok
User:  ok
ChatBot:  You can mail any problem you experience in this mail:  _______@gmail.com
User: 
User:  
ChatBot:  Mail for the support team: ________@gmail.com
User: bishal
User:  bishal
ChatBot:  Mail for the support team: ________@gmail.com
User: ok ok
User:  ok ok
ChatBot:  You can mail any problem you experience in this mail:  _______@gmail.com
User: hello
User:  hello
ChatBot:  Hey!
User: whaat are the course available
User:  whaat are the course available
ChatBot:  Accessing via a computer is recommended for the best experience.
User: okay
User:  okay
ChatBot:  Mail for the support team: ________@gmail.com
User: hi
User:  hi
ChatBot:  Hello
User: who are you ?
User:  who are you ?
ChatBot:  I'm NobelBot, your bot assi

KeyboardInterrupt: ignored